In [6]:
import requests

url = "http://localhost:4000/key/info"
key = "sk-d4eQY5xNDT-ye5jaR_QsLQ"
auth_token = "sk-1234"

# Constructing the headers
headers = {
    "Authorization": f"Bearer {auth_token}"
}

# Sending the GET request
response = requests.get(url, params={"key": key}, headers=headers)

# Printing the response
print(response.status_code)
print(response.json())

200
{'key': 'sk-d4eQY5xNDT-ye5jaR_QsLQ', 'info': {'key_name': 'sk-...QsLQ', 'key_alias': 'Agadji', 'soft_budget_cooldown': False, 'spend': 0.0, 'expires': None, 'models': [], 'aliases': {}, 'config': {}, 'user_id': None, 'team_id': None, 'permissions': {}, 'max_parallel_requests': None, 'metadata': {'team': 'AiTeam', 'user': 'Agadji', 'email': 'ag@gmail.com'}, 'blocked': None, 'tpm_limit': None, 'rpm_limit': None, 'max_budget': 70.0, 'budget_duration': '2m', 'budget_reset_at': '2024-12-17T10:39:07.386000+00:00', 'allowed_cache_controls': [], 'model_spend': {}, 'model_max_budget': {}, 'budget_id': None, 'created_at': '2024-12-17T08:45:44.479000+00:00', 'updated_at': '2024-12-17T10:37:47.653000+00:00', 'litellm_budget_table': None}}


In [ ]:
params = {
    "key": key,
    
}